<a href="https://colab.research.google.com/github/chupvl/gcolab/blob/main/2023_04_28_Talk_to_your_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Can we talk with the data? Tiny case for testing pandas_ai for human clearance data

This is a test of the [pandas_ai](https://github.com/gventuri/pandas-ai) package by [Gabriele Venturi](https://github.com/gventuri). 

Dataset I used is an AstraZeneca clearance data [CHEMBL3301370](https://www.ebi.ac.uk/chembl/assay_report_card/CHEMBL3301370/): ASTRAZENECA: Intrinsic clearance measured in human liver microsomes following incubation at 37C. Experimental range <3 to >150 microL/min/mg. Rapid Commun. Mass Spectrom. 2010, 24, 1730-1736.

**TLDR**: the proposed solution is far from optimal but it's working! Definitely an expertise layer.

## Libs

In [ ]:
!pip install pandasai -q

In [ ]:
!pip install astor -q

In [ ]:
!pip install python-dotenv -q

In [ ]:
!pip install openai -q

In [ ]:
import pandas as pd
from pandasai import PandasAI
from pandasai.llm.openai import OpenAI

In [ ]:
df = pd.DataFrame({
    "country": ["United States", "United Kingdom", "France", "Germany", "Italy", "Spain", "Canada", "Australia", "Japan", "China"],
    "gdp": [21400000, 2940000, 2830000, 3870000, 2160000, 1350000, 1780000, 1320000, 516000, 14000000],
    "happiness_index": [7.3, 7.2, 6.5, 7.0, 6.0, 6.3, 7.3, 7.3, 5.9, 5.0]
})


llm = OpenAI(api_token="sk-XXX")
pandas_ai = PandasAI(llm, verbose = True, conversational = False)
response = pandas_ai.run(df, "Calculate the sum of the gdp of north american countries")
print(response)

Running PandasAI with openai LLM...

Code generated:
```
north_america = ['United States', 'Canada', 'Mexico']
sum_gdp = df[df['country'].isin(north_america)]['gdp'].sum()
print(sum_gdp)
```


In [ ]:
response

23180000

## Human clearance example

In [ ]:
df_cl = pd.read_csv('/content/CHEMBL3301370.tsv', sep='\t')

In [ ]:
# cleaning because 
for i in ['AlogP', 'Molecular Weight', '#RO5 Violations']:
  df_cl[i] = pd.to_numeric(df_cl[i], errors='coerce').astype(float)
  df_cl = df_cl.dropna(subset=[i]).reset_index(drop=True)

In [ ]:
df_cl.columns

Index(['Molecule ChEMBL ID', 'Molecule Name', 'Molecule Max Phase',
       'Molecular Weight', '#RO5 Violations', 'AlogP', 'Compound Key',
       'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value',
       'Standard Units', 'pChEMBL Value', 'Data Validity Comment', 'Comment',
       'Uo Units', 'Ligand Efficiency BEI', 'Ligand Efficiency LE',
       'Ligand Efficiency LLE', 'Ligand Efficiency SEI', 'Potential Duplicate',
       'Assay ChEMBL ID', 'Assay Description', 'Assay Type', 'BAO Format ID',
       'BAO Label', 'Assay Organism', 'Assay Tissue ChEMBL ID',
       'Assay Tissue Name', 'Assay Cell Type', 'Assay Subcellular Fraction',
       'Assay Parameters', 'Assay Variant Accession', 'Assay Variant Mutation',
       'Target ChEMBL ID', 'Target Name', 'Target Organism', 'Target Type',
       'Document ChEMBL ID', 'Source ID', 'Source Description',
       'Document Journal', 'Document Year', 'Cell ChEMBL ID', 'Properties'],
      dtype='object')

### Provide molecule names for 5 top compounds that have highest LogP

In [ ]:
response = pandas_ai.run(df_cl, "Provide names for 5 compounds with the highest LogP")
response

979                                            FIBOFLAPON
734                                           MONTELUKAST
1055                                          PF-03715455
304     1,1',6,6',7,7'-HEXAHYDROXY-3,3'-DIMETHYL-N5-((...
1008                                          TELMISARTAN
Name: Molecule Name, dtype: object

In [ ]:
response = pandas_ai.run(df_cl, 
                         "Provide names for 5 compounds with the highest LogP",
                         is_conversational_answer=True)
response

"The top 5 compounds with the highest LogP are FIBOFLAPON, MONTELUKAST, PF-03715455, 1,1',6,6',7,7'-HEXAHYDROXY-3,3'-DIMETHYL-N5-(((2S)-3-((1H-tetrazol-5-yl)methyl)-2-(4-(2-methylpropoxy)phenyl)propyl)amino)carbonyl)-L-ornithine, and TELMISARTAN."

In [ ]:
response = pandas_ai.run(df_cl,
                         "Provide IDs for 5 compounds with the highest LogP")
response

979     CHEMBL1922660
734         CHEMBL787
1055    CHEMBL1938400
304     CHEMBL1094250
1008       CHEMBL1017
Name: Molecule ChEMBL ID, dtype: object

In [ ]:
response_man = df_cl.sort_values('AlogP', ascending=False)['Molecule ChEMBL ID'].to_list()[:5]
response_man

['CHEMBL1922660', 'CHEMBL787', 'CHEMBL1938400', 'CHEMBL1094250', 'CHEMBL1017']

### Provide ID for compounds that does not violate Lipinski rule and have highest LogP

In [ ]:
response = pandas_ai.run(df_cl, "Provide IDs for 5 compounds with the highest LogP that does not violate Lipinski rule")
response

257     CHEMBL2314383
1029    CHEMBL1917426
255      CHEMBL359806
917     CHEMBL1934415
69      CHEMBL1917422
Name: Molecule ChEMBL ID, dtype: object

In [ ]:
response_man = df_cl[df_cl['#RO5 Violations'] == 0].sort_values('AlogP', ascending=False)['Molecule ChEMBL ID'].to_list()[:5]
response_man

['CHEMBL2314383',
 'CHEMBL1917426',
 'CHEMBL359806',
 'CHEMBL1934415',
 'CHEMBL1917422']

### Provide ID for 5 top compounds that have highest LogP and lowest MW

In [ ]:
pandas_ai.run(df_cl, 
              "Provide IDs for 5 compounds with the highest LogP that does not violate Lipinski rule and MW < 300", 
              is_conversational_answer=True)

"The top 5 compounds with the highest LogP that don't break Lipinski's rule and have a molecular weight less than 300 are: CHEMBL1596896, CHEMBL669, CHEMBL550438, CHEMBL1644331, and CHEMBL139."

In [ ]:
pandas_ai.run(df_cl, 
              "Provide IDs for 5 compounds with the highest LogP that does not violate Lipinski rule and MW < 300", 
              is_conversational_answer=False)

287     CHEMBL1596896
34          CHEMBL669
965      CHEMBL550438
851     CHEMBL1644331
1082        CHEMBL139
Name: Molecule ChEMBL ID, dtype: object

### Provide ID for 5 top compounds that have highest LogP for human assay only

In [ ]:
df_cl['Assay Organism'].value_counts()

Homo sapiens    1097
Name: Assay Organism, dtype: int64

In [ ]:
response = pandas_ai.run(df_cl, "Provide IDs for compounds that does not violate Lipinski rule and have top 5 largest LogP but activity was measured for human only")
response

''

In [ ]:
response = pandas_ai.run(df_cl, "Provide IDs for compounds that does not violate Lipinski rule and have top 5 largest LogP for Homo sapiens organism only")
response

''

In [24]:
response = pandas_ai.run(df_cl, "Provide IDs for compounds that does not violate Lipinski rule and have top 5 largest LogP where assay organism is Homo sapiens")
response

Series([], Name: Molecule ChEMBL ID, dtype: object)

In [26]:
response = pandas_ai.run(df_cl, "Provide IDs for compounds where assay organism is human and LogP is between 2 and 7")
response

Series([], Name: Molecule ChEMBL ID, dtype: object)